<a href="https://colab.research.google.com/github/alexoman/IAVA/blob/main/Lab03_2022_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Strat Convolutional:
* torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

Strat de Pooling:
* torch.nn.MaxPool2d(kernel_size, stride=None, padding=0)
* torch.nn.AveragePool2d(kernel_size, stride=None, padding=0)

Strat de Adaptive Pool, intalnit adesea si ca Global Pool:
* torch.nn.AdaptiveAvgPool2d(output_size)
* torch.nn.AdaptiveMaxPool2d(output_size)

Strat de liniarizare:

* torch.nn.Flatten()



In [1]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizat pentru a reduce dimensiunea input-ului la una prestabilita, util cand marimea input-ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # hint: folositi stride si padding
*   [1, 10, 32, 32]
*   [1, 3, 2, 2]



In [24]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

### Completati codul pentru cerinta aici
layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(8,8), stride=(4,4), padding=(3,3))
print("Conv2 result shape",layer1(dummy_input_tensor).shape)
print()
layer2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(10,10), stride=(3,3), padding=(2,2))
print("Con2 result shape",layer2(dummy_input_tensor).shape)
print()
layer3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(10,10), stride=(50,50))
print("Conv2 result shape",layer3(dummy_input_tensor).shape)

layer4 = nn.MaxPool2d(kernel_size=(4,4)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer4(dummy_input_tensor).shape)



Conv2 result shape torch.Size([1, 10, 25, 25])

Con2 result shape torch.Size([1, 10, 32, 32])

Conv2 result shape torch.Size([1, 3, 2, 2])
Pool result shape torch.Size([1, 3, 25, 25])


## Instantierea seturilor de date

In [3]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [27]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def collate_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    normalized_tensor_image = normalized_tensor_image.unsqueeze(0)
    processed_images.append(normalized_tensor_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)
  
  return torch_images, torch_labels

batch_size=500
train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii
    * Liniarizati iesirea celui de-al doilea strat convolutional
    * Adaugat stratul final de tipul 'fully-connected'
    * Folositi o functie de activare la alegere

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [29]:
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    ### Completati codul pentru cerinta aici
    self.layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
    self.layer2 = nn.Conv2d(in_channels=10, out_channels=5, kernel_size=(3,3), stride=(2,2))
    self.flatten = nn.Flatten()
    self.linear = nn.Linear(245,10)
    self.activation = nn.ReLU()

  def forward(self,x):
    ### Completati codul pentru cerinta aici
    output_layer1 = self.layer1(x)
    output1 = self.activation(output_layer1)
    output_layer2 = self.layer2(output1)
    output2 = self.flatten(output_layer2)
    output2 = self.activation(output2)
    output_linear = self.linear(output2)
    output = self.activation(output_linear)
    return output

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Initializati numarul de epoci
  * Initializati retea
  * Initializati optimizator
  * Initializati functia de cost

In [30]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 5

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(), lr=1e-2,momentum=0.8)


"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie 
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [31]:
def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e + 1, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [32]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.2944154739379883
Acuratetea la finalul epocii 1 este 14.77%
Loss-ul la finalul epocii 1 are valoarea 2.224931478500366
Acuratetea la finalul epocii 2 este 19.53%
Loss-ul la finalul epocii 2 are valoarea 2.075087308883667
Acuratetea la finalul epocii 3 este 28.11%
Loss-ul la finalul epocii 3 are valoarea 2.0219709873199463
Acuratetea la finalul epocii 4 este 32.37%
Loss-ul la finalul epocii 4 are valoarea 1.9153759479522705
Acuratetea la finalul epocii 5 este 34.28%


## Reteaua LeNet

### Cerinte
  * **(2.5p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o


![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet


In [2]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    ### Completati codul pentru cerinta aici
    self.layer1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=(5,5), stride=(1,1))
    self.activation = nn.ReLU()# nn.GELU()
    self.avgPool1 = nn.AvgPool2d(kernel_size = (2,2), stride=(2,2), padding=0)
    self.layer2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=(1,1))
    # self.avgPool2 = nn.AveragePool2d(kernel_size = (2,2), stride=(2,2), padding=0)
    self.layer3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=(5,5), stride=(1,1))
    self.linear = nn.Linear(120,84)
    self.linear2 = nn.Linear(84,10)
    self.softmax = nn.Softmax(dim = 1)
    self.flatten = nn.Flatten()
    #self.norm1 = nn.BatchNorm2d(6)

  def forward(self,x):
    ### Completati codul pentru cerinta aici
    output_layer1 = self.layer1(x)
    output1 = self.activation(output_layer1)
    output_avgPool1 = self.avgPool1(output1)
    output2 = self.activation(output_avgPool1)
    #output2 = self.norm1(output2)
    output_layer2 = self.layer2(output2)
    output3 = self.activation(output_layer2)
    output_avgPool1 = self.avgPool1(output3)
    output4 = self.activation(output_avgPool1)
    
    output_layer3 = self.layer3(output4)
    output5 = self.activation(output_layer3)
    output5 = self.flatten(output5)
    output_linear = self.linear(output5)
    output6 = self.activation(output_linear)
    output_linear2 = self.linear2(output6)
    output7 = self.softmax(output_linear2)

    return output7

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * **(0.5p)** Redefiniti obiectele necesare pentru a antrena reteaua LeNet

In [4]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 8

# Definiti reteaua
lenet = LeNet().cuda()

# Definiti optimizatorul
lenet_optimizer = optim.Adam(lenet.parameters(), lr=1e-3)

# Completati aici codul pentru a face gradientii zero
lenet_optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [5]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

NameError: ignored

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* torch.nn.BatchNorm2d(num_features)
* torch.nn.InstanceNorm2d(num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de functii de activate:


* Relu
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu augmentarile mentionate mai sus in cadrul ultimei retele definite pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  
